# Advanced Geometry

When working on the pincell geometry before, we saw how to create cells that are composed of Boolean combinations from surface half-spaces. In principle, this is enough to create pretty much any geometry. However, there are a number of additional functions and classes in the Python API that can greatly simplify how a model is built. In this tutorial, we will leverage features available in the [`openmc.model`](https://docs.openmc.org/en/stable/pythonapi/model.html) module.

In [ ]:
import openmc

## Convenience Functions

There is one other function that can simplify the creation of pin-like geometries: `openmc.model.pin`. If you have a fuel pin that consists of multiple concentric cylinders, this function can put together a list of materials and surfaces and return back a fully constructed universe. Let's say we have a pin that consists of fuel, gap, clad, and water.

In [ ]:
fuel = openmc.Material()
fuel.add_element('U', 1.0)

clad = openmc.Material()
clad.add_element('Zr', 1.0)
clad.set_density('g/cm3', 5.0)

water = openmc.Material()
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)

Let's use a list comprehension to create some surfaces:

In [ ]:
radii = [0.35, 0.4, 0.45]
cyls = [openmc.ZCylinder(r=r) for r in radii]

Now we can create a full fuel pin by passing a list of materials and a list of surfaces that the materials will go between:

In [ ]:
fills = [fuel, None, clad, water]
fuel_pin = openmc.model.pin(cyls, fills)
type(fuel_pin)

In [ ]:
fuel_pin.plot(width=(2.0, 2.0))

## Composite Surfaces

When you create a surface in OpenMC, you get an instance of whatever class you specified. For example, if you call `openmc.XPlane(-5)`, you'll get an `XPlane` object. OpenMC also has several classes that act as "composite" surfaces. What this means is that they can be treated like normal surfaces such that you can use `-` and `+`, but the regions returned by these operators actually are composed from multiple primitive surfaces (planes, quadric, etc.). In this manner, they are equivalent to macrobodies in MCNP.

The currently available list of composite surfaces can be found [here](https://docs.openmc.org/en/stable/pythonapi/model.html#composite-surfaces).

### Rectangular Prism

In [ ]:
help(openmc.model.RectangularPrism)

As we see, this function creates an infinite rectangular prism for us so that we get all four sides without having to define each manually. Let's create a prism of width 10 cm and height 5 cm filled with our water material we created earlier.

In [ ]:
prism = openmc.model.RectangularPrism(10., 5.)
type(prism)

In [ ]:
cell = openmc.Cell(fill=water, region=-prism)
univ = openmc.Universe(cells=[cell])

univ.plot(width=(15., 15.))

We can even have rounded corners (fillets) on our prism if we want:

In [ ]:
prism = openmc.model.RectangularPrism(10., 5., corner_radius=1.)

cell = openmc.Cell(fill=water, region=-prism)
univ = openmc.Universe(cells=[cell])
univ.plot(width=(15., 15.))

### Hexagonal Prism

Along with rectangular prisms, you can also create a hexagonal prism with the `openmc.model.HexagonalPrism` class, which is very useful for many reactor types. For example, defining a typical hexagonal unit pin cell would look something like the following. Note that the `HexagonalPrism` requires us to tell it the length of the *side* of the hexagon.

In [ ]:
zcyl = openmc.ZCylinder(r = 1.0)
hex_prism = openmc.model.HexagonalPrism(2.0)

pin = openmc.Cell(fill=water, region=-zcyl)
outside_pin = openmc.Cell(region=+zcyl & -hex_prism)
univ = openmc.Universe(cells=[pin, outside_pin])
univ.plot(width=(5., 5.))

You can imagine that in this case, manually defining each of the six planes would be quite laborious! Similar to `RectangularPrism`, the `HexagonalPrism` class allows you to have rounded corners and also allows you to change the "orientation" of the hexagon (flats facing left-right or flats facing up-down).

In [ ]:
hex_prism = openmc.model.HexagonalPrism(2.0, orientation='y', corner_radius=0.5)

pin = openmc.Cell(fill=water, region=-zcyl)
outside_pin = openmc.Cell(region=+zcyl & -hex_prism)
univ = openmc.Universe(cells=[pin, outside_pin])
univ.plot(width=(5., 5.))

### Isogonal Octagon

As one specific example, the `IsogonalOctagon` composite surface allows you to model an octagon with normal surface-like operations:

In [ ]:
stop_sign = openmc.model.IsogonalOctagon((0., 0.), 1.0, 1.0)
cell = openmc.Cell(fill=water, region=-stop_sign)
univ = openmc.Universe(cells=[cell])
univ.plot(width=(3, 3))

## Other ways of creating surfaces

Sometimes you know what surface you want to define but figuring out all the appropriate coefficients for it can be challenging. For example, the `Plane` class normally requires that you specify the $A$, $B$, $C$, and $D$ coefficients, but it can be a pain to figure these out. Some surface classes in OpenMC have "alternate constructors" that can make creating the surface more intuitive. For `Plane` specifically, there is a `Plane.from_points` classmethod that allows you to provide three points that are on the plane, and it will figure out the appropriate coefficients:

In [ ]:
# Create a plane along the line x=y
xy_plane = openmc.Plane.from_points((0., 0., 0.), (1., 1., 0.), (0., 0., 1.))
xy_plane

Similarly, the `Cylinder` class has a `from_points` classmethod that allows you to specify two points going through the axis of the cylinder, which is extremely useful for defining non-axis-aligned cylinders:

In [ ]:
leaning_cylinder = openmc.Cylinder.from_points((0., 0., 0.), (1., 1., 0.), r=3.)
cell = openmc.Cell(fill=water, region=-leaning_cylinder)
univ = openmc.Universe(cells=[cell])
univ.plot(width=(10.0, 10.0))
univ.plot(width=(10.0, 10.0), basis='xz')

Another way that surfaces, particularly ones that are not axis-aligned, can be created is by using the `translate` and `rotate` methods that are available on all surfaces. Another way we could create the cylinder shown above is by creating an axis-aligned cylinder and then rotating it:

In [ ]:
cyl = openmc.YCylinder(r=3.)
rotated_cyl = cyl.rotate((0., 0., -45.))
cell = openmc.Cell(fill=water, region=-rotated_cyl)
univ = openmc.Universe(cells=[cell])
univ.plot(width=(10.0, 10.0))
univ.plot(width=(10.0, 10.0), basis='xz')

## Random sphere packing

OpenMC includes a few convenience functions for generating locations of randomly packed spheres that can be used to model TRISO particles and/or pebbles in a reactor core. To be clear, this capability is not a stochastic geometry capability like that included in MCNP. It's also important to note that OpenMC does not (yet) use delta tracking, which would normally speed up calculations in geometries with tons of surfaces and cells. However, the computational burden can be eased by placing random spheres in a lattice, which we will demonstrate here.

This capability relies on three functions/classes:
- `openmc.model.pack_spheres` -- generate locations of random spheres
- `openmc.model.TRISO` -- Cell-like object that holds a universe storing the internal structure of a pebble/TRISO
- `openmc.model.create_triso_lattice` -- Creates a lattice containing `TRISO` objects for improved tracking performance

Let's start with the `pack_spheres` function. This function takes an outer radius of the spheres, a containing region, and a packing fraction and will return an array of sphere coordinates. For our example, let's use spheres with a radius of 1 cm and a packing fraction of 30%. We'll put our spheres inside of a finite cylinder using another composite surface, `RightCircularCylinder`.

In [ ]:
r_cylinder = 10.
h_cylinder = 20.
cylinder = openmc.model.RightCircularCylinder(
    center_base=(0., 0., -10.), 
    height=h_cylinder,
    radius=r_cylinder
)

In [ ]:
r_sphere = 1.0
centers = openmc.model.pack_spheres(radius=r_sphere, region=-cylinder, pf=0.3, seed=1)
centers.shape

We see that with our cylindrical container, the `pack_spheres` function generated 449 locations for spheres. We can look at a few:

In [ ]:
centers[:10]

Now we need to actually creates cells for each of these spheres. To do so, we'll use the `TRISO` class. We'll need to define a universe that we want to fill each sphere. For a five-layer TRISO particle, this will be a universe of five annular spheres. Normally, we'd need to make four sphere surfaces and set up the regions between all of these surfaces. We can use another convenience function, `openmc.model.subdivide` to automatically set up these regions for us. Let's assume our TRISO model has five layers,

- $r < 0.5$: layer 1 
- $0.5 < r < 0.6$: layer 2
- $0.6 < r < 0.7$: layer 3
- $0.7 < r < 0.8$: layer 4
- $0.8 < r < 1.0$: layer 5

In order to make this example quicker, note that we're just filling these regions with the `water` and `clad` materials we created earlier, rather than using realistic materials for TRISO particles.

In [ ]:
triso_layer_r = [0.5, 0.6, 0.7, 0.8]
sphere_surfs = [openmc.Sphere(r=triso_r) for triso_r in triso_layer_r]

In [ ]:
regions = openmc.model.subdivide(sphere_surfs)
print(regions)

In [ ]:
fill_mats = [water, clad, None, water, clad]
cells = []
for i in range(len(regions)):
    cells.append(openmc.Cell(region=regions[i], fill=fill_mats[i]))

In [ ]:
sphere_univ = openmc.Universe(cells=cells)

Now we can create a `TRISO` object for each sphere center:

In [ ]:
trisos = [openmc.model.TRISO(outer_radius=r_sphere, 
                             fill=sphere_univ, center=center) for center in centers]

Each TRISO object actually is a Cell, in fact; we can look at the properties of the TRISO just as we would a cell:

In [ ]:
trisos[0]

Let's confirm that the packing fraction of our TRISOs is actually about 30%.

In [ ]:
from math import pi

volume_trisos = len(trisos)*4/3*pi*r_sphere**3
volume_cyl = pi * r_cylinder**2 * h_cylinder
volume_trisos / volume_cyl

Now that we have our spheres created, we can create a new universe that includes each TRISO as a cell plus a "background" cell that is composed of all space outside of the spheres. We will fill this background by graphite.

In [ ]:
graphite = openmc.Material()
graphite.add_element('C', 1.0)
graphite.set_density('g/cm3', 2.0)

In [ ]:
region_background = openmc.Intersection([~t.region for t in trisos])
background_cell = openmc.Cell(region=region_background & -cylinder, fill=graphite)
univ = openmc.Universe(cells=[background_cell] + trisos)
univ.plot(width=(20., 20.), pixels=[500, 500])

While this works in principle, it will lead to **very** poor tracking performance; every time a particle reaches the background cell, it has to determine the distance to the boundary of _every single_ sphere. To improve tracking performance, we can use the `create_triso_lattice` function to overlay a lattice that limits how many distance checks need to be performed.

In [ ]:
lower_left, upper_right = (-cylinder).bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape

lattice = openmc.model.create_triso_lattice(
    trisos=trisos,
    lower_left=lower_left,
    pitch=pitch,
    shape=shape,
    background=graphite
)

In [ ]:
main_cell = openmc.Cell(fill=lattice, region=-cylinder)
univ = openmc.Universe(cells=[main_cell])
univ.plot(width=(20., 20.), pixels=[500, 500])

In [ ]:
univ.plot(width=(20., 20.), pixels=[500, 500], color_by='material')